In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%matplotlib widget
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix

from ecephys.data import paths
from ecephys.utils.utils import load_df_h5
from ecephys.scoring import load_visbrain_hypnogram

In [6]:
pd.options.mode.chained_assignment = None  # default='warn'

## Read the data

In [7]:
condition = "BL"
subject = "Doppio"

hypnogram_path = Path(paths.hypnograms[condition][subject])
hypnogram = load_visbrain_hypnogram(hypnogram_path)

spw_path = Path(paths.spws[condition][subject])
with pd.HDFStore(spw_path) as store:
    spws, metadata = load_df_h5(store)
    
bl_hypno = hypnogram
spws_bl = spws

In [21]:
spws_bl["sink_amplitude"] = -spws_bl["sink_amplitude"]
spws_bl["sink_integral"] = -spws_bl["sink_integral"]

In [8]:
condition = "SR"
subject = "Doppio"

hypnogram_path = Path(paths.hypnograms[condition][subject])
hypnogram = load_visbrain_hypnogram(hypnogram_path)

spw_path = Path(paths.spws[condition][subject])
with pd.HDFStore(spw_path) as store:
    spws, metadata = load_df_h5(store)
    
rec_hypno = hypnogram
spws_rec = spws

In [61]:
spws_rec["sink_amplitude"] = -spws_rec["sink_amplitude"]
spws_rec["sink_integral"] = -spws_rec["sink_integral"]

## Look at basic summary statistics

In [62]:
bl_total_NREM = bl_hypno.loc[bl_hypno['state'] == 'N2'].duration.sum()
bl_total_NREM

3407.9599609375

In [63]:
rec_total_NREM = rec_hypno.loc[rec_hypno['state'] == 'N2'].duration.sum()
rec_total_NREM

4388.0

In [64]:
len(spws_bl) / bl_total_NREM

1.4715548473229179

In [65]:
len(spws_rec) / rec_total_NREM

1.1412944393801276

In [66]:
spws_bl.median()

start_time        3398.977382
end_time          3399.020182
duration             0.055200
midpoint          3398.998782
sink_amplitude       0.037043
sink_integral        1.348274
dtype: float64

In [67]:
spws_rec.median()

start_time        3755.848823
end_time          3755.948423
duration             0.045200
center_time       3755.898623
sink_amplitude       0.042923
sink_integral        1.427905
midpoint          3755.898623
dtype: float64

In [68]:
spws_bl.mean()

start_time        3500.968914
end_time          3501.033040
duration             0.064126
midpoint          3501.000977
sink_amplitude       0.040982
sink_integral        1.698988
dtype: float64

In [69]:
spws_rec.mean()

start_time        3683.488406
end_time          3683.541656
duration             0.053249
center_time       3683.515031
sink_amplitude       0.049645
sink_integral        1.953256
midpoint          3683.515031
dtype: float64

## Look at feature distributions and correlations

In [70]:
ax = scatter_matrix(spws_bl[["duration", "sink_amplitude", "sink_integral"]], figsize=(10, 10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
ax2 = scatter_matrix(spws_bl[["duration", "sink_amplitude", "sink_integral"]].applymap(np.log), figsize=(10, 10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [141]:
ax = scatter_matrix(spws_rec[["duration", "sink_amplitude", "sink_integral"]], figsize=(10, 10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
ax2 = scatter_matrix(spws_rec[["duration", "sink_amplitude", "sink_integral"]].applymap(np.log), figsize=(10, 10))

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/pandas/plotting/_matplotlib/tools.py:185: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(**fig_kw)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Look at log-normally distributed features

In [33]:
import scipy.stats as stats

def plot_qq(bl, rec, var):
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    
    stats.probplot(bl[var], dist="norm", plot=axes[0, 0])
    axes[0, 0].set_title("Baseline {}".format(var))
    stats.probplot(np.log(bl[var]), dist="norm", plot=axes[0, 1])
    axes[0, 1].set_title("Baseline log {}".format(var))
    stats.probplot(rec[var], dist="norm", plot=axes[1, 0])
    axes[1, 0].set_title("Recovery {}".format(var))
    stats.probplot(np.log(rec[var]), dist="norm", plot=axes[1, 1])
    axes[1, 1].set_title("Recovery log {}".format(var))

    plt.show()

In [36]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def plot_acf_pacf(bl, rec, var, lags=None):
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))

    plot_acf(bl[var], lags=lags, ax=axes[0, 0], title="Baseline {} acf".format(var))
    plot_pacf(bl[var], lags=lags, ax=axes[0, 1], title="Baseline {} pacf".format(var))
    plot_acf(rec[var], lags=lags, ax=axes[1, 0], title="Recovery {} acf".format(var))
    plot_pacf(rec[var], lags=lags, ax=axes[1, 1], title="Recovery {} pacf".format(var))

    plt.show()

In [44]:
def plot_vs_time(bl, rec, var):
    fig, axes = plt.subplots(2, 1)
    bl.plot.scatter(x='midpoint', y=var, ax=axes[0], title='Baseline', s=1)
    rec.plot.scatter(x='midpoint', y=var, ax=axes[1], title='Recovery', s=1)
    plt.show()

In [41]:
def test_slope(spws, x, y):
    return stats.linregress(spws[x], spws[y])

In [48]:
# https://dfrieds.com/math/effect-size.html
def interpret_cohens_d(cohens_d):
    """
    Determines text interpretation of effect size given Cohen's d value

    :param cohens_d: float of Cohen's d value
    :returns: effect_size_interpretation: adjective to describe magnitude of effect size
    """
    if 0<=cohens_d<0.1:
        effect_size_interpretation = "Very Small"
    elif 0.1<=cohens_d<0.35:
        effect_size_interpretation = "Small"
    elif 0.35<=cohens_d<0.65:
        effect_size_interpretation = "Medium"
    elif 0.65<=cohens_d<0.9:
        effect_size_interpretation = "Large"
    elif cohens_d >= 0.9:
        effect_size_interpretation = "Very Large"
    return effect_size_interpretation

In [49]:
def two_sample_data_setup_for_cohens_d(mean_1, mean_2, sample_std_dev_1, sample_std_dev_2):
    """
    Creates two normal distributions of data values. 
    Calculates differences in values and standard deviation of list of differences.
    Calculates Cohen's D - equation used in dependent t-tests

    :param mean_1: mean of group one
    :param mean_2: mean of group two
    :param sample_std_dev_1: sample standard deviation of group one
    :param sample_std_dev_2: sample standard deviation of group two
    
    :returns: values_1, values_2, cohens_d: values for group one, values for group two, Cohen's d
    """
    values_1 = np.random.normal(loc=mean_1, scale=sample_std_dev_1, size=3000)
    values_2 = np.random.normal(loc=mean_2, scale=sample_std_dev_2, size=3000)
    differences_group_values = np.subtract(values_1, values_2)
    std_dev_differences_group_values = np.std(differences_group_values, ddof=1)
    cohens_d = round(abs((mean_1 - mean_2)/std_dev_differences_group_values), 2)
    return values_1, values_2, cohens_d

In [145]:
import seaborn as sns

def visualize_distributions(values_1, values_2, cohens_d, cohens_d_interpretation, xlabel):
    """
    Plots KDE plots of values from values_1 and values_2.
    Inserts Cohen's d value and interpretation in title

    :param values_1: values in group one
    :param values_2: values in group two 
    :param cohens_d: Cohen's d value
    :param cohens_d_interpretation: text to describe magnitude of effect size
    :returns: plot figure
    """
    plt.figure(figsize=(10, 8))
    sns.distplot(values_1, hist=False, label='Baseline')
    sns.distplot(values_2, hist=False, label='Recovery')
    plt.xlabel(xlabel, labelpad=13)
    plt.ylabel("frequency", labelpad=13)
    plt.legend()
    plt.title("Distribution of Two Groups with {0} Cohen's d of {1}".format(cohens_d_interpretation, cohens_d), y=1.015, fontsize=22)

### Duration

In [75]:
plot_qq(spws_bl, spws_rec, 'duration')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
plot_acf_pacf(spws_bl, spws_rec, 'duration')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [77]:
plot_vs_time(spws_bl, spws_rec, 'duration')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
test_slope(spws_bl, 'midpoint', 'duration')

LinregressResult(slope=-4.373680046369506e-07, intercept=0.06565694397878907, rvalue=-0.026955067218466595, pvalue=0.05629578191887162, stderr=2.2908671601796585e-07)

In [79]:
test_slope(spws_rec, 'midpoint', 'duration')

LinregressResult(slope=-6.195370772066635e-07, intercept=0.05553154135535736, rvalue=-0.0408319605972056, pvalue=0.0038518251999304738, stderr=2.1426898169156522e-07)

In [80]:
stats.ttest_ind(np.log(spws_bl.duration), np.log(spws_rec.duration), equal_var=False)

Ttest_indResult(statistic=20.65269723280505, pvalue=8.045621452491604e-93)

In [146]:
values_bl, values_rec, cohens_d = two_sample_data_setup_for_cohens_d(np.log(spws_bl.duration).mean(), 
                                                                     np.log(spws_rec.duration).mean(), 
                                                                     np.log(spws_bl.duration).std(), 
                                                                     np.log(spws_rec.duration).std())
visualize_distributions(np.log(spws_bl.duration), np.log(spws_rec).duration, cohens_d, interpret_cohens_d(cohens_d), xlabel="log transformed sink duration")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [82]:
stat, p, med, tbl = stats.median_test(spws_bl.duration, spws_rec.duration) 
print(stat, p, med)

283.2734566919725 1.4530071033944713e-63 0.05039994931780711


In [83]:
stats.kruskal(spws_bl.duration, spws_rec.duration)

KruskalResult(statistic=420.0513454828952, pvalue=2.378320153860877e-93)

In [84]:
stats.mannwhitneyu(spws_bl.duration, spws_rec.duration)

MannwhitneyuResult(statistic=9588992.0, pvalue=1.189244412133681e-93)

### Sink integral

In [85]:
plot_qq(spws_bl, spws_rec, 'sink_integral')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [86]:
plot_acf_pacf(spws_bl, spws_rec, 'sink_integral')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
plot_vs_time(spws_bl, spws_rec, 'sink_integral')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
test_slope(spws_bl, 'midpoint', 'sink_integral')

LinregressResult(slope=-1.5230166424420831e-05, intercept=1.7523089509510328, rvalue=-0.026913521826331065, pvalue=0.05667668732439738, stderr=7.98965303646123e-06)

In [89]:
test_slope(spws_rec, 'midpoint', 'sink_integral')

LinregressResult(slope=-6.410611209351652e-05, intercept=2.1893922944597723, rvalue=-0.08192378788949, pvalue=6.404693019157619e-09, stderr=1.1022540256987766e-05)

In [142]:
stats.ttest_ind(np.log(spws_bl.sink_integral), np.log(spws_rec.sink_integral), equal_var=False)

Ttest_indResult(statistic=-6.380293571775402, pvalue=1.8468545438930396e-10)

In [147]:
values_bl, values_rec, cohens_d = two_sample_data_setup_for_cohens_d(np.log(spws_bl.sink_integral).mean(), 
                                                                     np.log(spws_rec.sink_integral).mean(), 
                                                                     np.log(spws_bl.sink_integral).std(), 
                                                                     np.log(spws_rec.sink_integral).std())
visualize_distributions(np.log(spws_bl.sink_integral), np.log(spws_rec).sink_integral, cohens_d, interpret_cohens_d(cohens_d), xlabel="log-transformed sink integral")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [92]:
stat, p, med, tbl = stats.median_test(spws_bl.duration, spws_rec.duration) 
print(stat, p, med)

283.2734566919725 1.4530071033944713e-63 0.05039994931780711


In [93]:
stats.kruskal(spws_bl.duration, spws_rec.duration)

KruskalResult(statistic=420.0513454828952, pvalue=2.378320153860877e-93)

In [94]:
stats.mannwhitneyu(spws_bl.duration, spws_rec.duration)

MannwhitneyuResult(statistic=9588992.0, pvalue=1.189244412133681e-93)

## Look at non-normally distributed features

### Sink amplitude

In [95]:
plot_acf_pacf(spws_bl, spws_rec, 'sink_amplitude')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [96]:
plot_vs_time(spws_bl, spws_rec, 'sink_amplitude')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [97]:
test_slope(spws_bl, 'midpoint', 'sink_amplitude')

LinregressResult(slope=-1.1055941119353796e-07, intercept=0.04136933781774623, rvalue=-0.017324171830455957, pvalue=0.21996308499049494, stderr=9.012166373781214e-08)

In [98]:
test_slope(spws_rec, 'midpoint', 'sink_amplitude')

LinregressResult(slope=-9.925463936367327e-07, intercept=0.05330111781665446, rvalue=-0.10638483069404586, pvalue=4.4162680965860146e-14, stderr=1.311155070674533e-07)

In [99]:
stats.mannwhitneyu(spws_bl.sink_amplitude, spws_rec.sink_amplitude)

MannwhitneyuResult(statistic=8606885.0, pvalue=4.135189919070619e-164)

In [140]:
plt.figure(figsize=(10, 8))
sns.distplot(spws_bl.sink_amplitude, hist=False, label='Baseline')
sns.distplot(spws_rec.sink_amplitude, hist=False, label='Recovery')
plt.xlabel("Sink ampliutde (mA/mm)", labelpad=13)
plt.ylabel("frequency", labelpad=13)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


## Look at spw density

In [101]:
plt.figure(figsize=(10, 8))
sns.distplot(bl_hypno[bl_hypno.state == "N2"].duration, hist=False, label='Baseline')
sns.distplot(rec_hypno[rec_hypno.state == "N2"].duration, hist=False, label='Recovery')
plt.xlabel("NREM Episode Duration", labelpad=13)
plt.ylabel("frequency", labelpad=13)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [104]:
def get_spw_density(hypno, spws):  
    N2 = hypno[hypno.state == "N2"]
    N2['n_spws'] = N2.apply(lambda episode: np.sum(np.logical_and(spws.midpoint >= episode.start_time, spws.midpoint <= episode.end_time)), axis=1)
    N2['spw_density'] = N2.apply(lambda episode: episode.n_spws / episode.duration, axis=1)
    return N2

In [105]:
bl_density = get_spw_density(bl_hypno, spws_bl)

plt.figure()
ax = bl_density.spw_density.plot.hist(bins=20)

print("Mean spw density: ", bl_density.spw_density.mean(), "\n", "Median spw density: ", bl_density.spw_density.median())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean spw density:  1.140861231105374 
 Median spw density:  1.181469298245614


In [106]:
ax = bl_density.plot(x='start_time', y='spw_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [107]:
test_slope(bl_density, 'start_time', 'spw_density')

LinregressResult(slope=-8.02965520318897e-07, intercept=1.1436586658025978, rvalue=-0.007193029090727634, pvalue=0.9588302242229545, stderr=1.548004336418494e-05)

In [108]:
rec_density = get_spw_density(rec_hypno, spws_rec)

plt.figure()
ax = rec_density.spw_density.plot.hist(bins=20)

print("Mean spw density: ", rec_density.spw_density.mean(), "\n", "Median spw density: ", rec_density.spw_density.median())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean spw density:  0.8953429318500221 
 Median spw density:  0.9666666666666667


In [109]:
plt.figure()
rec_density.boxplot(column=['spw_density'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [110]:
ax = rec_density.plot(x='start_time', y='spw_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [111]:
test_slope(rec_density, 'start_time', 'spw_density')

LinregressResult(slope=-2.3658373107345113e-06, intercept=0.9034761536944171, rvalue=-0.0197433718568858, pvalue=0.8952043934432364, stderr=1.785963725509652e-05)

In [112]:
plot_qq(bl_density, rec_density, 'spw_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [113]:
stats.ttest_ind(bl_density.spw_density, rec_density.spw_density, equal_var=False)

Ttest_indResult(statistic=4.986017842510567, pvalue=2.81748183770641e-06)

In [114]:
values_bl, values_rec, cohens_d = two_sample_data_setup_for_cohens_d(bl_density.spw_density.mean(), 
                                                                     rec_density.spw_density.mean(), 
                                                                     bl_density.spw_density.std(), 
                                                                     rec_density.spw_density.std())
visualize_distributions(bl_density.spw_density, rec_density.spw_density, cohens_d, interpret_cohens_d(cohens_d))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [115]:
stat, p, med, tbl = stats.median_test(bl_density.spw_density, rec_density.spw_density) 
print(stat, p, med)

12.236767448508168 0.00046856936917419225 1.0416666666666667


In [116]:
stats.kruskal(bl_density.spw_density, rec_density.spw_density)

KruskalResult(statistic=20.013423367815168, pvalue=7.690043496790038e-06)

In [117]:
stats.mannwhitneyu(bl_density.spw_density, rec_density.spw_density)

MannwhitneyuResult(statistic=612.0, pvalue=3.906741821608331e-06)

## Epoched spw density

In [122]:
# https://stackoverflow.com/questions/21160134/flatten-a-column-with-value-of-type-list-while-duplicating-the-other-columns-va
def unnest_df(df, col, reset_index=False):
    import pandas as pd
    col_flat = pd.DataFrame([[i, x] 
                       for i, y in df[col].apply(list).iteritems() 
                           for x in y], columns=['I', col])
    col_flat = col_flat.set_index('I')
    df = df.drop(col, 1)
    df = df.merge(col_flat, left_index=True, right_index=True)
    if reset_index:
        df = df.reset_index(drop=True)
    return df

def _get_spw_density(hypno, spws, epoch_length):  
    N2 = hypno[np.logical_and(hypno.state == "N2", hypno.duration >= epoch_length)]
    N2['epoch_start_times'] = N2.apply(lambda episode: np.arange(episode.start_time, episode.end_time - epoch_length, epoch_length), axis=1)
    N2 = unnest_df(N2, 'epoch_start_times')
    N2 = N2.rename(columns={'end_time': 'episode_end_time', 'start_time': 'epsiode_start_time', 'duration': 'episode_duration', 'epoch_start_times': 'start_time'})
    N2['end_time'] = N2.apply(lambda epoch: epoch.start_time + epoch_length, axis=1)
    N2['n_spws'] = N2.apply(lambda epoch: np.sum(np.logical_and(spws.midpoint >= epoch.start_time, spws.midpoint <= epoch.end_time)), axis=1)
    N2['spw_density'] = N2.apply(lambda epoch: epoch.n_spws / epoch_length, axis=1)
    return N2

In [123]:
epoch_length = 10

In [124]:
bl_density = _get_spw_density(bl_hypno, spws_bl, epoch_length)

plt.figure()
ax = bl_density.spw_density.plot.hist()

print("Mean spw density: ", bl_density.spw_density.mean(), "\n", "Median spw density: ", bl_density.spw_density.median(), "\n", "Number of spws: ", len(bl_density))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean spw density:  1.1983818770226535 
 Median spw density:  1.2 
 Number of spws:  309


In [125]:
ax = bl_density.plot(x='start_time', y='spw_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [126]:
test_slope(bl_density, 'start_time', 'spw_density')

LinregressResult(slope=-1.6610652263795806e-05, intercept=1.2579259015967363, rvalue=-0.0894132622875766, pvalue=0.11676103251782523, stderr=1.0560209827671667e-05)

In [127]:
rec_density = _get_spw_density(rec_hypno, spws_rec, epoch_length)

plt.figure()
ax = rec_density.spw_density.plot.hist()

print("Mean spw density: ", rec_density.spw_density.mean(), "\n", "Median spw density: ", rec_density.spw_density.median(), "\n", "Number of spws: ", len(rec_density))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean spw density:  1.0744471744471744 
 Median spw density:  1.1 
 Number of spws:  407


In [128]:
ax = rec_density.plot(x='start_time', y='spw_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [129]:
test_slope(rec_density, 'start_time', 'spw_density')

LinregressResult(slope=1.3265239515761145e-05, intercept=1.0244531927042564, rvalue=0.06620749371949933, pvalue=0.1825183849478493, stderr=9.934053483395669e-06)

In [130]:
plot_qq(bl_density, rec_density, 'spw_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/numpy/lib/function_base.py:2474: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


In [131]:
stats.ttest_ind(bl_density.spw_density, rec_density.spw_density, equal_var=False)

Ttest_indResult(statistic=4.192199485028918, pvalue=3.141771371051424e-05)

In [155]:
values_bl, values_rec, cohens_d = two_sample_data_setup_for_cohens_d(bl_density.spw_density.mean(), 
                                                                     rec_density.spw_density.mean(), 
                                                                     bl_density.spw_density.std(), 
                                                                     rec_density.spw_density.std())
visualize_distributions(bl_density.spw_density, rec_density.spw_density, cohens_d, interpret_cohens_d(cohens_d), xlabel="Event density (per sec)")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [133]:
stat, p, med, tbl = stats.median_test(bl_density.spw_density, rec_density.spw_density) 
print(stat, p, med)

5.967595922034474 0.014571132954603964 1.1


In [134]:
stats.kruskal(bl_density.spw_density, rec_density.spw_density)

KruskalResult(statistic=13.727795859574371, pvalue=0.00021130399223254876)

In [135]:
stats.mannwhitneyu(bl_density.spw_density, rec_density.spw_density)

MannwhitneyuResult(statistic=52756.0, pvalue=0.0001057282844640674)

### Sensitivity Analysis

In [136]:
def _get_density_ttest_pval(epoch_length):
    bl_density = _get_spw_density(bl_hypno, spws_bl, epoch_length)
    rec_density = _get_spw_density(rec_hypno, spws_rec, epoch_length)
    return stats.ttest_ind(bl_density.spw_density, rec_density.spw_density, equal_var=False).pvalue

def _get_density_MannWhitney_pval(epoch_length):
    bl_density = _get_spw_density(bl_hypno, spws_bl, epoch_length)
    rec_density = _get_spw_density(rec_hypno, spws_rec, epoch_length)
    return stats.mannwhitneyu(bl_density.spw_density, rec_density.spw_density).pvalue

def _get_density_nEpochs(hypno, spws, epoch_length):
    bl_density = _get_spw_density(hypno, spws_bl, epoch_length)
    rec_density = _get_spw_density(rec_hypno, spws_rec, epoch_length)
    return stats.mannwhitneyu(bl_density.spw_density, rec_density.spw_density).pvalue

df = pd.DataFrame(data={'epoch_length': np.arange(5, 65, 5)})
df['ttest_pval'] = df.apply(lambda x: _get_density_ttest_pval(x.epoch_length), axis=1)
df['mw_pval'] = df.apply(lambda x: _get_density_MannWhitney_pval(x.epoch_length), axis=1)
df['n_epochs_bl'] = df.apply(lambda x: len(_get_spw_density(bl_hypno, spws_bl, x.epoch_length)), axis=1)
df['n_epochs_rec'] = df.apply(lambda x: len(_get_spw_density(rec_hypno, spws_rec, x.epoch_length)), axis=1)

In [137]:
df

,epoch_length,ttest_pval,mw_pval,n_epochs_bl,n_epochs_rec
0,5,0.000003,0.000008,651,845
1,10,0.000031,0.000106,309,407
2,15,0.000323,0.000173,197,268
3,20,0.000287,0.000463,143,192
4,25,0.000792,0.001399,107,150
5,30,0.000529,0.000762,83,124
6,35,0.000407,0.000422,72,104
7,40,0.004949,0.006501,60,86
8,45,0.004413,0.002861,49,76
9,50,0.001822,0.002163,42,66


In [138]:
df[['ttest_pval', 'mw_pval', 'epoch_length']].plot(x='epoch_length')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='epoch_length'>

In [139]:
df[['n_epochs_bl', 'n_epochs_rec', 'epoch_length']].plot(x='epoch_length')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='epoch_length'>